## **Walidacja Danych | Obsługa Wyjątków | Kody Statusu**

### **Prosta Klasa Danych**

In [2]:
class Book:
    """Represents a book with a title, author, and category."""

    def __init__(self, id, title, author, description, rating):
        self.id = id
        self.title = title
        self.author = author
        self.description = description
        self.rating = rating


BOOKS = [
    Book(
        id=1,
        title="The Great Gatsby",
        author="F. Scott Fitzgerald",
        description="A classic novel about the American Dream.",
        rating=4,
    ),
    Book(
        id=2,
        title="Don Quixote",
        author="Miguel de Cervantes",
        description="A Spanish novel about what it means to be a knight.",
        rating=5,
    ),
]

Podobnie jak poprzednio, klasa danych `Book` posłuży do celów testowych w FastAPI.

### **Pydantic**

Biblioteka `pydantic` jest używana do modelowania danych, parsowania danych i wydajnej obsługi błędów. Jest powszechnie używana jako źródło informacji na temat walidacji danych i sposobu obsługi danych przychodzących do aplikacji FastAPI.

In [3]:
from pydantic import BaseModel, Field


class RequestBook(BaseModel):
    id: int | None = Field(description="Unique identifier for the book. Optional", default=None)
    title: str = Field(min_length=1, max_length=50)
    author: str = Field(min_length=1, max_length=50)
    description: str = Field(min_length=10, max_length=100)
    rating: int = Field(ge=1, le=5)

    model_config = {
        "json_schema_extra": {
            "example": {
                "title": "The Great Gatsby",
                "author": "F. Scott Fitzgerald",
                "description": "A classic novel about the American Dream.",
                "rating": 4,
            }
        }
    }

W powyższym przykładzie tworzymy model książki obsługujący walidację danych przy użyciu biblioteki `pydantic`. Następnie w funkcji tworzącej nową książkę konwertujemy model obsługujący walidację na rzeczywistą instancję książki. 

Atrybut `model_config` zapewnia dodatkową informację nt. oczekiwanych wartości.

In [4]:
from fastapi import FastAPI

app = FastAPI()


@app.post("/create-book")
async def create_book(book_request: RequestBook) -> None:
    book = Book(**book_request.model_dump())
    book.id = create_unique_id()
    BOOKS.append(book)


def create_unique_id() -> int:
    return 1 + max((book.id for book in BOOKS), default=0)

### **Walidacja Danych Ścieżki & Zapytań**

Możemy zapewnić walidację danych dla parametru ścieżki za pomocą funkcji `Path`.

In [5]:
from fastapi import Path, Query


@app.get("/book/{book_id}")
async def get_book(book_id: int = Path(ge=0, description="Unique identifier for the book.")):
    for book in BOOKS:
        if book.id == book_id:
            return book


@app.get("/books/")
async def get_book_by_rating(rating: int = Query(ge=1, le=5, description="Rating of the book.")):
    return list(book for book in BOOKS if book.rating == rating)

### **Kody Statusu**

Kod statusu HTTP jest używany, aby pomóc klientowi, który jest systemem lub użytkownikiem przesyłającym dane do serwera, zrozumieć co się stało w aplikacji po stronie serwera. Kody statusu są międzynarodowymi standardami określającymi, w jaki sposób klient lub serwer powinien obsługiwać wynik żądania. Dzięki temu każdy, kto wysyła żądanie, wie, czy jego przesłanie powiodło się, czy nie.

Najbardziej powszechne kody:
- Seria `1xx` - Odpowiedzi informacyjne, że żądanie jest przetwarzane. Może to być informacja dla użytkownika, że coś dzieje się za kulisami.
- Seria `2xx` - Udane żądania.
- Seria `3xx` - Odczytywane jako wskazówki, co oznacza, że dalsze działania muszą zostać zakończone.
- Seria `4xx` - Błędy spowodowane przez klienta.
- Seria `5xx` - Błędy serwera.


Powszechne kody `2xx` - udane żądania:
- `200` - OK. Jest standardową odpowiedzią na pomyślne żądanie. Powszechnie używana w przypadku pomyślnych żądań GET, gdy dane są zwracane do klienta.
- `201` - Created. Oznacza to, że żądanie powiodło się i utworzono nowy zasób. Jest to często używane, gdy żądanie POST tworzy nową jednostkę dla aplikacji.
- `204` - No content. Oznacza, że zakończyło się ono powodzeniem, ale nie utworzyło encji, ani odpowiedź nie niczego nie zwraca. Jest powszechnie używany do żądań PUT.

Powszechne kody `4xx` - błędy klienta:
- `400` - Bad request. Oznacza to, że nie może przetworzyć żądania z powodu błędu klienta. Powszechnie używane w przypadku nieprawidłowych metod żądań.
- `401` - Unauthorized. Oznacza to, że klient nie ma ważnego uwierzytelnienia dla zasobu docelowego.
- `404` - Not found. Oznacza, że nie można znaleźć zasobów żądanych przez klienta.
- `422` - Unprocessable entity. Oznacza to, że w żądaniu klienta występują jakieś błędy semantyczne.

Powszechne kody `5xx` - błędy serwera:
- `500` - Internal server error. Jest to ogólny komunikat o błędzie, gdy wystąpił nieoczekiwany błąd na serwerze.